In [1]:
# coding: utf-8
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import nltk
import re
import os
import math
import json
import sys
import time
import json
from PorterStemmer import *
#from nltk.stem import PorterStemmer
#from nltk.tokenize import word_tokenize
ps = PorterStemmer()
from tqdm.notebook import tqdm

In [2]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [3]:
delim = '''[ ',(){}.:;"`\n]'''

In [4]:
data_dir = "C:\Files\col764-a2-release\col764-ass2-release"

In [5]:
metadata = os.path.join(data_dir,"metadata.csv")

In [6]:
meta_df = pd.read_csv(metadata)
meta_df = meta_df[['cord_uid','title','abstract','pdf_json_files','pmc_json_files']]
#meta_df = meta_df.head(1000)
meta_df['pdf_json_files'] = meta_df['pdf_json_files'].astype(str)
meta_df['pmc_json_files'] = meta_df['pmc_json_files'].astype(str)

c:\users\kshitij alwadhi\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
meta_df.head()

,cord_uid,title,abstract,pdf_json_files,pmc_json_files
0,ug7v899j,Clinical features of culture-proven Mycoplasma...,OBJECTIVE: This retrospective chart review des...,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json
1,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...,Inflammatory diseases of the respiratory tract...,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json
2,ejv2xln0,Surfactant protein-D and pulmonary host defense,Surfactant protein-D (SP-D) participates in th...,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json
3,2b73a28n,Role of endothelin-1 in lung disease,Endothelin-1 (ET-1) is a 21 amino acid peptide...,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json
4,9785vg6d,Gene expression in epithelial cells in respons...,Respiratory syncytial virus (RSV) and pneumoni...,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json


In [11]:
def getTokensFromTextProcess(textarr):
    res = []
    for text in textarr:
        words = re.split(delim, text)
        for w in words:
            if(len(w) > 0):
                #temp = ps.stem(w.lower(), 0, len(w)-1)
                temp = w.lower()
                if temp not in stop_words:
                    res.append(temp)
    return res

In [26]:
def get_idf_dict_new():
    idf_dict = {}
    filemap = {}
    for row in tqdm(meta_df.itertuples(),total = meta_df.shape[0]):
        uid = row.cord_uid
        alltext = []
        if row.pmc_json_files:
            paths = row.pmc_json_files
        elif row.pdf_json_files:
            paths = row.pdf_json_files
        filemap[uid] = paths
        for path in paths.split("; "):
            if(path == "nan"):
                continue
            datapath = os.path.join(data_dir,path)
            with open(datapath) as data:
                data_dict = json.load(data)
                for section in data_dict['body_text']:
                    section_text = section['text']
                    alltext.append(section_text)
        tokens = getTokensFromTextProcess(alltext)
        tokens = set(tokens)
        for token in tokens:
            if token not in idf_dict:
                idf_dict[token]=1
            else:
                idf_dict[token]+=1
    return idf_dict,filemap        

In [27]:
idf_dict,filemap = get_idf_dict_new()

KeyboardInterrupt: 